This is the CUDA parallel code for calculating the Dunn index, according to the definition of José María Luna-Romera (BD-Dunn)

In [8]:
import pandas as pd

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
da = pd.read_csv('/content/drive/MyDrive/Faculdade./TCC./VDU./Aplicando CUDA no VDU./Algoritmo LUNA./Data/luna_k5_f20_5000000.dat')

In [15]:
nome_arquivo = 'luna_k5_f20_5000000.dat'

# Salvando o DataFrame em um arquivo de texto
da.to_csv(nome_arquivo, sep=' ', index=False)

In [16]:
# Abrir o arquivo .dat em modo de leitura
with open('luna_k5_f20_5000000.dat', 'r') as arquivo:
    # Ler o conteúdo do arquivo
    conteudo = arquivo.read()

    # Remover as aspas utilizando a função replace()
    conteudo_sem_aspas = conteudo.replace('"', '')

# Abrir o arquivo em modo de escrita e escrever o conteúdo sem as aspas
with open('luna_k5_f20_5000000.dat', 'w') as arquivo:
    arquivo.write(conteudo_sem_aspas)

In [1]:
# Check the CPU available and the gcc version
!lscpu | grep CPU
!g++ -v

CPU op-mode(s):                  32-bit, 64-bit
CPU(s):                          2
On-line CPU(s) list:             0,1
CPU family:                      6
Model name:                      Intel(R) Xeon(R) CPU @ 2.00GHz
CPU MHz:                         2000.136
NUMA node0 CPU(s):               0,1
Using built-in specs.
COLLECT_GCC=g++
COLLECT_LTO_WRAPPER=/usr/lib/gcc/x86_64-linux-gnu/9/lto-wrapper
OFFLOAD_TARGET_NAMES=nvptx-none:hsa
OFFLOAD_TARGET_DEFAULT=1
Target: x86_64-linux-gnu
Configured with: ../src/configure -v --with-pkgversion='Ubuntu 9.4.0-1ubuntu1~20.04.1' --with-bugurl=file:///usr/share/doc/gcc-9/README.Bugs --enable-languages=c,ada,c++,go,brig,d,fortran,objc,obj-c++,gm2 --prefix=/usr --with-gcc-major-version-only --program-suffix=-9 --program-prefix=x86_64-linux-gnu- --enable-shared --enable-linker-build-id --libexecdir=/usr/lib --without-included-gettext --enable-threads=posix --libdir=/usr/lib --enable-nls --enable-clocale=gnu --enable-libstdcxx-debug --enable-libstdcxx-t

In [2]:
#Check the GPU available and the nvcc version
!nvidia-smi
!nvcc --version

Mon Jul 10 19:04:34 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
# Install nvcc4jupter to easily work with CUDA file and install the nvcc plugin
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-17uexs_g
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-17uexs_g
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4287 sha256=db55c1ae250a1453173c0562b4bd1768f5276d605bc8ad063d7b600a153db32c
  Stored in directory: /tmp/pip-ephem-wheel-cache-xelxu8a7/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


In [17]:
%%cu
// This is the CUDA program for Dunn index calculation
#include <stdio.h>
#include <stdlib.h>
#include <float.h>
#include <math.h>

//luna_k5_f20_5000 -- MP:5120, BS:128, MB:MP/BS=40
//luna_k5_20_10000 -- MP:16384, BS:128, MB:MP/BS=128
//luna_k5_20_25000 -- MP:32768, BS:128, MB:MP/BS=256
//luna_k5_20_50000 -- MP:65536, BS:128, MB:MP/BS=512
//luna_k5_f20_500000 -- MP:524288, BS:128, MB:MP/BS=4096
//luna_k5_20_2500000 -- MP:4194304, BS:128, MB:MP/BS=32768
//luna_k5_20_5000000 -- MP:8388608, BS:128, MB:MP/BS=65536
// Test_k2_f2_10, Iris_k3_f4_150 - Digits_k10_f64_1797 - Electricity_k2_f8_45311
// MAX_POINTS 2048 (others) or 65536 (Electric)
// NF 2 (Test) 4 (Iris) 64 (Digits) 8 (Electricity)
// BLOCK_SIZE 128 (others) 64 (Digits)
#define MAX_POINTS 8388608
#define NF 20
#define BLOCK_SIZE 128

#define MAX_CLUSTERS 20
#define MAX_BLOCKS 65536

// Calculate the euclidian distance between two points in the CPU
float distance(float v1[], float v2[]) {
    float sum = 0.0;
    for (int d = 0; d < NF; d++) {
        sum += pow((v1[d] - v2[d]), 2);
    }
    return sqrt(sum);
}

// Calculate the euclidian distance between two points in the GPU
__device__ float distance(float *s_point, int p, int q, int nfeat) {
    float sum = 0.0;
    for (int d = 0; d < nfeat; d++) {
        sum = sum + pow((s_point[p+d] - s_point[q+d]), 2);
    }
    return sqrt(sum);
}

// Kernel that calculates the parcial sums (in each dimenstion) of the instances coordenates
__global__ void centroids(int cluster, float *d_centroid_tmp, float *d_point, int *d_cluster_start, int nfeat) {

  __shared__ float s_centroid[BLOCK_SIZE * NF];
  int tid = threadIdx.x;
  int lower = d_cluster_start[cluster];
  int i = (blockIdx.x * blockDim.x) + threadIdx.x;
  int size = d_cluster_start[cluster+1] - d_cluster_start[cluster];
  int p; //reduction step: 64, 32, 16, 8, 4, 2,1

  // All threads initialize with zero the shared memory
  for (int d = 0; d < nfeat; d++) {
    s_centroid[tid * nfeat + d] = 0.0;
  }
  __syncthreads();

  // Copy points from global memory to shared memory
  if (i < size) {
    for (int d = 0; d < nfeat; d++) {
      s_centroid[tid * nfeat + d] = (float ) d_point[(lower + i)*nfeat + d];
    }
  }
  __syncthreads();

  // Perform a local reduction on the memory shared data
  // It starts with 64 threads, then 32, 16, 8, 4, 2, 1
  p = blockDim.x / 2;
  while (p != 0) {
    if (tid < p) {
	    for (int d = 0; d < nfeat; d++) {
        s_centroid[tid*nfeat+d] = s_centroid[tid*nfeat+d] + s_centroid[(tid+p)*nfeat+d];
      }
    }
    __syncthreads();
    p = p/2;
  }

  // Thread zero of each block moves the local result to the global memory
  if (tid == 0) {
    for (int d = 0; d < nfeat; d++) {
        d_centroid_tmp[blockIdx.x * nfeat + d] = (float )s_centroid[d];
    }
  }
}

// Kernel that finds the cluster/point with the greatest distance from the centroid
__global__ void maxs_intra(int cluster, int *d_index_tmp, float *d_maxs_tmp, float *d_centroid, float *d_point, int *d_cluster_start, int nfeat) {

  __shared__ float s_point[(BLOCK_SIZE+1)*NF];
  __shared__ int s_pos[(BLOCK_SIZE+1)];
  int tid = threadIdx.x;
  int lower = d_cluster_start[cluster];
  int i = (blockIdx.x * blockDim.x) + threadIdx.x;
  int size = d_cluster_start[cluster+1] - d_cluster_start[cluster];
  int nb, d, p, r, q;
  float dist;

  // All threads initialize shared memory
  s_pos[tid] = lower + tid;
  for (d = 0; d < nfeat; d++) {
    s_point[tid*nfeat+d] = 0.0;
  }
  __syncthreads();

  // Copy data points from global memory to shared memory
  if (i < size) {
    for (d = 0; d < nfeat; d++) {
      s_point[tid*nfeat+d] = d_point[(lower+i)*nfeat+d];
    }
  }

  // store centroid in the last position of the vector shared memory to save memory
  if (tid == 0) {
    for (d = 0; d < nfeat; d++) {
      s_point[blockDim.x*nfeat+d] = d_centroid[cluster*nfeat+d];
    }
  }
  __syncthreads();

  // adjust limit for the last block
  if (blockIdx.x == (gridDim.x -1)) {
    nb = size % blockDim.x;
  } else {
    nb = blockDim.x;
  }

  // each thread calculates dist
  if (tid < nb) {
    r = tid*nfeat; // point index
    q = blockDim.x*nfeat; // centroid index
    dist = distance(s_point, r, q, nfeat);
    s_point[tid*nfeat] = dist;
  }
  __syncthreads();

  // reduction to find the maximum distance
  p = blockDim.x / 2; // log steps
  while (p != 0) {
    if (tid < p) {
      if (s_point[tid*nfeat] < s_point[(tid+p)*nfeat]) {
        s_point[tid*nfeat] = s_point[(tid+p)*nfeat];
        s_pos[tid] = s_pos[tid+p];
      }
    }
    __syncthreads();
    p = p/2;
  }

  // Thread zero of each block copy data to glocal memory
  if (tid == 0) {
    d_index_tmp[blockIdx.x] = s_pos[0];
    d_maxs_tmp[blockIdx.x] = s_point[0];
  }
}

int main()
{
  int num_clusters; // number of clusters
  int cluster_size[MAX_CLUSTERS]; //cluster sizes
  static float point[MAX_POINTS][NF]; // cluster data
  float *d_point; // GPU cluster data
  float centroid[MAX_CLUSTERS][NF]; // centroid data
  float *d_centroid; // GPU centroid data
  float centroid_tmp[MAX_BLOCKS][NF]; // centroid temporary data
  float centroid_g[NF];
  float *d_centroid_tmp; // GPU centroid temporary data
  int index_tmp[MAX_BLOCKS]; // index temporary data
  int *d_index_tmp; // GPU index temporary data
  float maxs_tmp[MAX_BLOCKS]; // max values temporary
  float *d_maxs_tmp; // GPU max values temporary
  int cluster_start[MAX_CLUSTERS+1]; // start cluster indexes
  int *d_cluster_start; // GPU start cluster indexes
  FILE *fp; // file pointer
  int size = 0; // total number of points
  int nfeat; // number of attributes
  clock_t start, stop; // measure time
  float running_time; // running time
  int nblocks; // number of blocks
  int cluster; // current cluster
  float sum; // sum of elements
  float dist; // distance
  float max_distance; // maximum distance
  float min_distance; // minimum distance
  int cluster1; // cluster chosen
  int p1; // index chosen

  // Input the number of clusters and the cluster information
  // Format: 1st line: #clusters #features, 2nd: cluster sizes, 3rd: data



  //fp = fopen("luna_k5_f20_5000.dat", "r");
  //fp = fopen("luna_k5_f20_10000.dat", "r");
  //fp = fopen("luna_k5_f20_25000.dat", "r");
  //fp = fopen("luna_k5_f20_50000.dat", "r");
  //fp = fopen("luna_k5_f20_500000.dat", "r");
  //fp = fopen("luna_k5_f20_2500000.dat", "r");
  fp = fopen("luna_k5_f20_5000000.dat", "r");
  //fp = fopen("test_k2_f2_10.dat", "r");
  //fp = fopen("iris_k3_f4_150.dat", "r");
  //fp = fopen("digits_k10_f64_1797.dat", "r");
  //fp = fopen("electricity_k2_f8_45311.dat", "r");


  // Read file (upload file first if running in Collab)
  fscanf(fp, "%d %d", &num_clusters, &nfeat);
  for (int k = 0; k < num_clusters; k++) {
    fscanf(fp, "%d", &cluster_size[k]);
    size = size + cluster_size[k];
  }
  for (int i = 0; i < size; i++) {
    for (int j = 0; j < nfeat; j++) {
       fscanf(fp, "%f", &point[i][j]);

       }
  }
  fclose(fp);

  // prefix sum to find out the beginning of each cluster
  cluster_start[0] = 0;
  for (int i = 1; i < num_clusters+1; i++) {
    cluster_start[i] = cluster_start[i-1] + cluster_size[i-1];
  }

   // initialize global centroid with zero
  for (int j = 0; j < nfeat; j++) {
    centroid_g[j] = 0.0;
  }

  // Allocate GPU memory
  cudaMalloc(&d_cluster_start, (MAX_CLUSTERS+1)*sizeof(int));
  cudaMalloc(&d_point, MAX_POINTS*NF*sizeof(float));
  cudaMalloc(&d_centroid_tmp, MAX_BLOCKS*NF*sizeof(float));
  cudaMalloc(&d_index_tmp, MAX_BLOCKS*sizeof(int));
  cudaMalloc(&d_maxs_tmp, MAX_BLOCKS*sizeof(float));
  cudaMalloc(&d_centroid, MAX_CLUSTERS*NF*sizeof(float));

  // start clock to measure running time
  start = clock();

  // Copy data (cluster points and start indices) to the GPU
  cudaMemcpy(d_point, point, MAX_POINTS*nfeat*sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_cluster_start, cluster_start, (MAX_CLUSTERS+1)*sizeof(int), cudaMemcpyHostToDevice);

  //tam = 0;
  // find centroids: launch the kernel for each cluster
  for (cluster = 0; cluster < num_clusters; cluster++) {

    // Number of blocks is size of cluster divided by the block size
    nblocks = (cluster_size[cluster] + BLOCK_SIZE - 1) / BLOCK_SIZE;


    // launch kernel and verify if got any error
    centroids<<<nblocks, BLOCK_SIZE>>>( cluster, d_centroid_tmp, d_point, d_cluster_start, nfeat );
    cudaError_t error = cudaGetLastError();
    if(error != cudaSuccess) { printf("CUDA error: %s\n", cudaGetErrorString(error)); exit(-1); }

    // Wait for the kernel to finish and copy centroid temporary data for the host (CPU)
    // The kernel returns the parcial sums of each block
    cudaDeviceSynchronize();
    cudaMemcpy(&centroid_tmp, d_centroid_tmp, MAX_BLOCKS*NF*sizeof(float), cudaMemcpyDeviceToHost);
    cudaDeviceSynchronize();


    // Calculate centroid and store it in the centroid_tmp
    // The parcial sums need to be accumulated and divided by the cluster size
    for (int i = 0; i < nfeat; i++) {
      sum = 0.0;
      for (int j = 0; j < nblocks; j++) {
        sum = sum + centroid_tmp[j][i];
      }
      centroid_g[i] = centroid_g[i] + sum;
      centroid_tmp[0][i] = sum / (float )cluster_size[cluster];
      centroid[cluster][i] = centroid_tmp[0][i];
    }
    //tam=tam+(float )cluster_size[cluster];
  }


  printf("\nCentroid Global: ");
  for (int i = 0; i < nfeat; i++) {
     centroid_g[i] = centroid_g[i] / size;
     printf("%.2f ", centroid_g[i]);
  }

  // Copy centroids to the GPU
  cudaMemcpy(d_centroid, centroid, MAX_CLUSTERS*NF*sizeof(float), cudaMemcpyHostToDevice);

  // Find the global centroid and store at centroid[MAX_CLUSTERS][NF]
  // Initialize global centroid with zero
  for (int k = 0; k < nfeat; k++) {
    centroid[MAX_CLUSTERS][k] = 0.0;
  }
  for (int k = 0; k < nfeat; k++) {
    for (int i = 0; i < num_clusters; i++) {
      centroid[MAX_CLUSTERS][k] = centroid[MAX_CLUSTERS][k] + centroid[i][k];
    }
  }
  for (int k = 0; k < nfeat; k++) {
    centroid[MAX_CLUSTERS][k] = centroid[MAX_CLUSTERS][k] / num_clusters;
  }

  // Find the centroid closer to the global centroid
  min_distance = DBL_MAX;
  for (int i = 0; i < num_clusters; i++) {
    for (int k = 0; k < nfeat; k++) {
      dist = distance(centroid[i], centroid_g);
      if (dist < min_distance) {
        min_distance = dist;
      }
    }
  }
  // Now min_distance is the numerator of the Dunn index

  // Now, find maximum diameter launching the kernel for each cluster again
  max_distance = 0;
  for (cluster = 0; cluster < num_clusters; cluster++) {

    // Number of blocks is size of cluster divided by the block size
    nblocks = (cluster_size[cluster] + BLOCK_SIZE - 1) / BLOCK_SIZE;

    // launch kernel and verify if got any error
    maxs_intra<<<nblocks, BLOCK_SIZE>>>( cluster, d_index_tmp, d_maxs_tmp, d_centroid, d_point, d_cluster_start, nfeat );
    cudaError_t error = cudaGetLastError();
    if(error != cudaSuccess) { printf("CUDA error: %s\n", cudaGetErrorString(error)); exit(-1); }

    // Wait for the kernel to finish and copy maximum temporary data for the host (CPU)
    // The kernel returns the several maximums, one for each block
    cudaDeviceSynchronize();
    cudaMemcpy(&maxs_tmp, d_maxs_tmp, MAX_BLOCKS*sizeof(float), cudaMemcpyDeviceToHost);
    cudaMemcpy(&index_tmp, d_index_tmp, MAX_BLOCKS*sizeof(int), cudaMemcpyDeviceToHost);
    cudaDeviceSynchronize();

    // Calculate the global maximum and store it in the max_distance
    // The parcial maximums need to be compared and the global maximum stored
    // The cluster and the maximum point position need to be saved (cluster1 and p1)
    for (int j = 0; j < nblocks; j++) {
      if (maxs_tmp[j] > max_distance) {
        max_distance = maxs_tmp[j];
        p1 = index_tmp[j];
        cluster1 = cluster;
      }
    }
  }

  // finalize runtime calculation
  stop = clock();

  // Print results
  printf("\nMin intercluster %.2f", min_distance);
  printf("\nMax intracluster %.2f", max_distance);
  printf("\nThe Dunn index: %.2f", min_distance / max_distance);

  // Print the time taken
  running_time = (float)(stop - start) / CLOCKS_PER_SEC;
  printf("\nTime taken: %lf milissegundos\n", 1000.0*running_time);

  // Free GPU memory
  cudaFree( d_cluster_start );
  cudaFree( d_point );
  cudaFree( d_centroid_tmp );
  cudaFree( d_index_tmp );
  cudaFree( d_maxs_tmp );
  cudaFree( d_centroid );

  return 0;
}


Centroid Global: 0.75 0.55 0.35 0.65 0.45 0.55 0.45 0.55 0.65 0.45 0.55 0.65 0.45 0.45 0.65 0.45 0.65 0.45 0.45 0.55 
Min intercluster 0.96
Max intracluster 0.41
The Dunn index: 2.36
Time taken: 408.757001 milissegundos

